In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [53]:
#change the path accordingly
df=pd.read_csv('/kaggle/input/fake-news/train.csv')
df.head()

In [55]:
#remove the rows which has nan values present in a column
print(df.isna().sum())
print(df.shape)
print("---"*5)
for i,j in df.isna().sum().items():
    print(f'{i :{10}}   {j/df.shape[0]}')
print("---"*5)
df=df.dropna()
print(df.isna().sum())
print(df.shape)
    


In [56]:
df.head(2)

In [57]:
df.shape

In [58]:
#for computational constraints
df=df.iloc[:10000,:]
print(df.shape)

In [59]:
x=df[['title','author','text']]
print(x.shape)
y=df['label']
print(y.shape)

In [60]:
#Exploratory data analysis
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.countplot(x=df['label'])

ax.bar_label(container=ax.containers[0], labels=['reliable','unreliable'])
plt.show()

#1: unreliable
#0: reliable


In [61]:
#author description
df['author'].describe()

# Top 10 author's whose news are more reliable and top 10 authors whose news are not at all reliable


In [62]:
from collections import Counter
reliable_author=list(df[df['label']==1]['author'])
print(type(Counter(reliable_author)))
reliable_author_dict=Counter(reliable_author)
reliable_author_dict=sorted(reliable_author_dict.items(), key=lambda i: i[1], reverse=True)[:10]
print("Top 10 reliable author's   :",reliable_author_dict)
print("---"*10)
unreliable_author=list(df[df['label']==0]['author'])
print(type(Counter(unreliable_author)))
unreliable_author_dict=Counter(unreliable_author)
unreliable_author_dict=sorted(unreliable_author_dict.items(), key=lambda i: i[1], reverse=True)[:10]
print("Top 10 unreliable author's   :",unreliable_author_dict)



**Some statistics related to  the text**

In [63]:
df['text'].describe()

In [64]:
#top words present in the text
list1=[]
for i in df['text']:
    for j in i.split():
        list1.append(j)
words_freq=Counter(list1)
words_top10=sorted(words_freq.items(), key=lambda i: i[1], reverse=True)[:50]
print("Top 10 words in texts   :",words_top10)

# thus removing stop words is mandatory because most of the top words present are stopwords
    


In [13]:
df.columns

In [65]:
# data preprocessing 
df['combine']=df['title'] + " "+df['author']+ " " + df['text']
df.head(3)

In [67]:
df=df[['label','combine']].reset_index()
df=df.iloc[:,1:]
df.head()

In [68]:
text=df['combine'].copy()
text.head(5)

In [69]:
import nltk
import re
from nltk.corpus import stopwords


In [70]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
print(STOPWORDS)

In [71]:
text1="Jalal eating eat good best 0=== engineer "
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

corpus = []
for i in range(0, 1):
    review = re.sub('[^a-zA-Z]', ' ', text1).strip().lower()

    
    review = [ps.stem(word) for word in review.split() if not word in stopwords.words('english')]
    print(review)
    review = [lemmatizer.lemmatize(word) for word in review ]
    print(review)
    review = ' '.join(review)
    corpus.append(review)
corpus

In [72]:
len(text)

In [ ]:
#stemming and lemmatization
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

corpus = []
for i in range(0, len(text)):
    review = re.sub('[^a-zA-Z]', ' ', text[i]).strip().lower()
    
    review = [word for word in review.split() if not word in stopwords.words('english')]
    #add lemmatizaion with stemming
    review = [lemmatizer.lemmatize(word) for word in review ]
    
    review = ' '.join(review)

    print(i)
    print("----")
    
    corpus.append(review)
print(len(corpus))

In [110]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from sklearn.metrics import confusion_matrix

In [76]:
voc_size=5000

In [128]:
onehot_representation=[one_hot(words,voc_size)for words in corpus] 
print(len(onehot_representation))

sent_length=50
embedded_docs=pad_sequences(onehot_representation,padding='post',maxlen=sent_length)
print(len(embedded_docs))

In [129]:
print(onehot_representation[0])

In [130]:
## Creating model and add two lstm layers
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


In [103]:
import numpy as np
X=np.array(embedded_docs).copy()
y=np.array(list(df['label']))
print(X.shape,y.shape)

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [132]:
#fit the model
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=128)
history

In [138]:
#plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [133]:
y_pred=model.predict(X_test)
y_pred

In [135]:
#function to predict class
def predict_class(y_pred):
    pred_list=[]
    for i in y_pred:
        if i >0.5:
            pred_list.append(1)
        else:
            pred_list.append(0)
    return pred_list
        
        
prediction=predict_class(y_pred)   
confusion_matrix(y_test,prediction)

In [136]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,prediction)